Idea of this notebook:
- separate keys with process_audio_files function
- ideally for both datasets (practical_dl and MKA)
- remember optimal tresholds and do some majority voting in future data loader

save separated audio in folder structure:
- processed data
    - train/val/test
        - label 
            - segmented recordings
- later add there also files from https://www.kaggle.com/datasets/nguyncaoduy/keystroke-noiseless-final

Idea for DataLoader:
- Separate audio for 10ms frames
- Calculate energylevel in each frame, if the treshold is exceeded then enlargen the frame with subsequent 90ms or so

In [ ]:
import os
from tqdm import tqdm
import shutil
from sklearn.model_selection import train_test_split

from src.utils.data_engineering_utils import process_audio_files

### `MKA` and `Practical_dl` datasets preprocessing

In [ ]:
OUTPUT_DIR = "./data/final/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_DIR_IMG = "./data/final_img/"
os.makedirs(OUTPUT_DIR_IMG, exist_ok=True)

DATA_DIR = './data/raw_data/MKA/Mac/Raw Data/'
DATASET = 'mka'

process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)

DATA_DIR = './data/raw_data/practical_dl/'
DATASET = 'practical'
process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)


In [ ]:
DATA_DIR = './data/raw_data/custom_mac/'
DATASET = 'custom_mac'

process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)

Processing files in : 100%|██████████| 64/64 [00:20<00:00,  3.19it/s]


In [ ]:
DATA_DIR = './data/raw_data/custom_dishwasher/'
DATASET = 'custom_dishwasher'

process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)

Processing files in : 100%|██████████| 63/63 [00:17<00:00,  3.61it/s]


In [ ]:
DATA_DIR = './data/raw_data/custom_washingmachine/'
DATASET = 'custom_washing_machine'

process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)

Processing files in : 100%|██████████| 63/63 [00:15<00:00,  4.10it/s]


In [2]:
DATA_DIR = './data/raw_data/custom_open_window/'
DATASET = 'custom_open_window'

process_audio_files(DATA_DIR, OUTPUT_DIR, OUTPUT_DIR_IMG, DATASET, True)

Processing files in : 100%|██████████| 63/63 [00:17<00:00,  3.67it/s]


### `Noiseless` Dataset

In [ ]:
SOURCE_DIR = './data/raw_data/data_noiseless_final/'
DATA_DIR_NOISELESS = './data/raw_data/noiseless/'

labels_in_dir_new = set()
for dirname, _, filenames in os.walk(SOURCE_DIR):
    for filename in filenames:
        if filename.endswith('.wav'):
            label = filename.split('-')[1]
            label_dir = os.path.join(DATA_DIR_NOISELESS, label)
            if not os.path.exists(label_dir):
                os.makedirs(label_dir)

            src_path = os.path.join(dirname, filename)
            filename_new = 'mac_live_' + label + '_' + filename.split('-')[-1].split('.')[0] + '.wav'
            dst_path = os.path.join(label_dir, filename_new)

            shutil.copy(src_path, dst_path)

In [11]:
def move_files(file_paths, subset_name):
    for file_path in file_paths:
        label = file_path.split('/')[-2]

        if not os.path.exists(os.path.join(OUTPUT_DIR, subset_name, 'noiseless', label)):
            os.makedirs(os.path.join(OUTPUT_DIR, subset_name, 'noiseless', label), exist_ok=True)

        file_name = os.path.basename(file_path)
        dest_path = os.path.join(OUTPUT_DIR, subset_name, 'noiseless', label, file_name)

        shutil.move(file_path, dest_path)

In [ ]:
for key in os.listdir(DATA_DIR_NOISELESS):
    key_dir = os.path.join(DATA_DIR_NOISELESS, key)
    if os.path.isdir(key_dir):
        files = [os.path.join(key_dir, f) for f in os.listdir(key_dir) if f.endswith('.wav')]
        train_files, temp_files = train_test_split(files, test_size=0.3, random_state=42)
        val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)

        move_files(train_files, 'train')
        move_files(val_files, 'val')
        move_files(test_files, 'test')